In [2]:
import requests
from datetime import datetime

num_results = 100

# Query URL with adjustable result count
query_url = f"https://gis.summitcountyco.gov/arcgis/rest/services/ParcelQueryTool/SummitMap1_Pro321/MapServer/19/query?where=SOURCE=1&orderByFields=MODDATE%20DESC&resultRecordCount={num_results}&outFields=*&returnGeometry=false&f=json"

# Send the request
response = requests.get(query_url)
if response.status_code == 200:
    data = response.json()
    features = data.get("features", [])
    print(f"{num_results} Most Recently Modified Parcels:\n")
    for feature in features:
        attributes = feature.get("attributes", {})
        object_id = attributes.get("OBJECTID")
        ppi = attributes.get("PPI")
        mod_date = attributes.get("MODDATE")
        mod_type = attributes.get("MODTYPE")
        # Convert MODDATE (Unix timestamp in milliseconds) to readable date
        if mod_date:
            mod_date = datetime.fromtimestamp(mod_date / 1000).strftime("%Y-%m-%d %H:%M:%S")
        #print(f"Parcel ID: {object_id}, PPI: {ppi}, Last Modified: {mod_date}")
        print(f"Object ID: {object_id}")
        print(f"PPI: {ppi}")
        print(f"MOD Date: {mod_date}")
        print(f"MOD Type: {mod_type}\n")
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")

100 Most Recently Modified Parcels:

Object ID: 367
PPI: 1697-3440-00-014
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 714
PPI: 1951-0610-00-016
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 728
PPI: 1951-0610-00-011
MOD Date: 2025-05-26 18:00:00
MOD Type: 0

Object ID: 731
PPI: 1951-0630-00-008
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 11034
PPI: 2097-3540-00-078
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 11129
PPI: 2093-3540-00-037
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 11212
PPI: 2097-3540-00-074
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 11242
PPI: 2097-3540-00-074
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 16102
PPI: 2211-3143-10-009
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 17902
PPI: 9999-9999-00-049
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 19860
PPI: 2375-1310-00-001
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Object ID: 19921
PPI: 2375-1310-00-001
MOD Date: 2025-05-26 18:00:00


In [3]:
import requests
from datetime import datetime

def get_database_ppis():
    """Retrieve all PPIs from the database (layer 12) for comparison"""
    url = "https://gis.summitcountyco.gov/arcgis/rest/services/ParcelQueryTool/SummitMap1_Pro321/MapServer/12/query"
    
    all_features = []
    batch_size = 1000
    offset = 0
    params = {
        "where": "1=1",
        "outFields": "*",  # Get all fields for matching features
        "f": "json",
        "returnGeometry": "false",
        "outSR": "102654",
        "resultRecordCount": batch_size
    }
    
    print("Loading database PPIs for comparison...")
    
    try:
        while True:
            params["resultOffset"] = offset
            
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            data = response.json()
            
            if "features" not in data or not data["features"]:
                break
                
            all_features.extend(data["features"])
            
            if len(data["features"]) < batch_size:
                break
                
            offset += batch_size
        
        # Create a dictionary mapping PPI to feature data
        ppi_to_feature = {}
        for feature in all_features:
            ppi = feature.get("attributes", {}).get("PPI")
            if ppi is not None:
                ppi_to_feature[ppi] = feature
        
        print(f"Loaded {len(ppi_to_feature)} unique PPIs from database")
        return ppi_to_feature
        
    except requests.exceptions.RequestException as e:
        print(f"Error querying the database: {e}")
        return {}

def main():
    num_results = 100
    
    # Get database PPIs for comparison
    database_ppis = get_database_ppis()
    
    if not database_ppis:
        print("Failed to load database PPIs. Exiting.")
        return
    
    # Query URL with adjustable result count
    query_url = f"https://gis.summitcountyco.gov/arcgis/rest/services/ParcelQueryTool/SummitMap1_Pro321/MapServer/19/query?where=SOURCE=1&orderByFields=MODDATE%20DESC&resultRecordCount={num_results}&outFields=*&returnGeometry=false&f=json"
    
    # Send the request
    response = requests.get(query_url)
    if response.status_code == 200:
        data = response.json()
        features = data.get("features", [])
        
        print(f"\n{num_results} Most Recently Modified Parcels:")
        print("="*60)
        
        matching_count = 0
        
        for i, feature in enumerate(features, 1):
            attributes = feature.get("attributes", {})
            object_id = attributes.get("OBJECTID")
            ppi = attributes.get("PPI")
            mod_date = attributes.get("MODDATE")
            mod_type = attributes.get("MODTYPE")
            
            # Convert MODDATE (Unix timestamp in milliseconds) to readable date
            if mod_date:
                mod_date_readable = datetime.fromtimestamp(mod_date / 1000).strftime("%Y-%m-%d %H:%M:%S")
            else:
                mod_date_readable = "N/A"
            
            # Check if PPI exists in database
            if ppi in database_ppis:
                matching_count += 1
                database_feature = database_ppis[ppi]
                database_attributes = database_feature.get("attributes", {})
                
                print(f"\n[{i}] ✓ MATCH FOUND")
                print(f"Object ID: {object_id}")
                print(f"PPI: {ppi}")
                print(f"MOD Date: {mod_date_readable}")
                print(f"MOD Type: {mod_type}")
                
                print(f"\nMatching Database Feature:")
                print(f"  OBJECTID: {database_attributes.get('OBJECTID')}")
                print(f"  SubName: {database_attributes.get('SubName')}")
                print(f"  FullStreet: {database_attributes.get('FullStreet')}")
                print(f"  TownName: {database_attributes.get('TownName')}")
                print(f"  OwnerAdd1: {database_attributes.get('OwnerAdd1')}")
                print(f"  TotAcres: {database_attributes.get('TotAcres')}")
                print(f"  YearBuilt: {database_attributes.get('YearBuilt')}")
                print("-" * 60)
            else:
                print(f"\n[{i}] ✗ NO MATCH")
                print(f"Object ID: {object_id}")
                print(f"PPI: {ppi}")
                print(f"MOD Date: {mod_date_readable}")
                print(f"MOD Type: {mod_type}")
                print("-" * 60)
        
        # Summary
        print(f"\nSUMMARY:")
        print(f"Total queried parcels: {len(features)}")
        print(f"Matching PPIs found in database: {matching_count}")
        print(f"Non-matching PPIs: {len(features) - matching_count}")
        print(f"Match percentage: {(matching_count/len(features)*100):.1f}%")
        
    else:
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")

if __name__ == "__main__":
    main()

Loading database PPIs for comparison...
Loaded 21031 unique PPIs from database

100 Most Recently Modified Parcels:

[1] ✓ MATCH FOUND
Object ID: 367
PPI: 1697-3440-00-014
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Matching Database Feature:
  OBJECTID: 19851
  SubName: METES & BOUNDS (ALL)
  FullStreet: 
  TownName: Unincorporated Summit County
  OwnerAdd1: 6060 BROADWAY
  TotAcres: 82.11
  YearBuilt: 0
------------------------------------------------------------

[2] ✓ MATCH FOUND
Object ID: 714
PPI: 1951-0610-00-016
MOD Date: 2025-05-26 18:00:00
MOD Type: 4

Matching Database Feature:
  OBJECTID: 18316
  SubName: MEADOWBROOK ACRES SUB
  FullStreet: Meadowbrook LN (CR 1378)
  TownName: Unincorporated Summit County
  OwnerAdd1: 110 MOSS WAY
  TotAcres: 1.098
  YearBuilt: 1964
------------------------------------------------------------

[3] ✓ MATCH FOUND
Object ID: 728
PPI: 1951-0610-00-011
MOD Date: 2025-05-26 18:00:00
MOD Type: 0

Matching Database Feature:
  OBJECTID: 19217
  SubNa